In [140]:
# from google.colab import drive
# drive.mount('/content/drive')

- 데이터 경로 변수

In [141]:
# DATA_PATH = "/content/drive/MyDrive/파이널프로젝트/data/"
# DATA_PATH

In [142]:
DATA_PATH = "data/"
DATA_PATH

'data/'

- 시드값

In [143]:
SEED = 42

- 데이터 불러오기

In [166]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

- 공통 피처 파일 불러오기

In [167]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 2310), (12225, 2310))

# 결측치 처리

In [168]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

pivot_지점코드_A112000    2
pivot_지점코드_A144000    2
pivot_지점코드_A202000    2
pivot_지점코드_A373000    2
pivot_중분류_DC캐주얼       2
                     ..
Tuesday_sum           2
Wednesday_sum         2
방문일수_대비_구매건수          2
구매주기_대비_구매금액          2
방문당_평균구매금액            2
Length: 2170, dtype: int64

In [172]:
train_tr[train_tr["ID"] == 'train_137']

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격
517742,train_137,2005-04-25 19:30:00,A112000,패션잡화,수입종합화장품,5173,-155000
517747,train_137,2005-04-25 19:32:00,A112000,패션잡화,수입종합화장품,5173,155000
517749,train_137,2005-04-25 19:33:00,A112000,패션잡화,수입종합화장품,5173,155000


In [171]:
train_ft[train_ft["pivot_지점코드_A112000"].isnull() == True]

,ID,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,...,전체_성수기_구매비율,전체_준성수기_구매비율,전체_중간기_구매비율,전체_비수기_구매비율,성수기_구매집중도,계절_구매변동성,구매주기_대비_구매금액,방문당_평균구매금액,구매금액_대비_환불금액,환불건수_대비_구매건수
137,train_137,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,-inf,inf
2594,train_2594,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,-inf,inf


In [169]:
mask = test_ft.isnull().sum() > 0
test_ft.isnull().sum()[mask]

pivot_지점코드_A112000    3
pivot_지점코드_A144000    3
pivot_지점코드_A202000    3
pivot_지점코드_A373000    3
pivot_중분류_DC캐주얼       3
                     ..
Tuesday_sum           3
Wednesday_sum         3
방문일수_대비_구매건수          3
구매주기_대비_구매금액          3
방문당_평균구매금액            3
Length: 2170, dtype: int64

# 특성 공학(Feature Engineering)

- ID 변수 제외

In [148]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 2309), (12225, 2309))

- 추가 피처 만들어 보기

In [149]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_지점코드_") ]

train_ft["지점코드별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["지점코드별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["지점코드별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["지점코드별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["지점코드별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["지점코드별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2312), (12225, 2312))

In [150]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_중분류_") ]

train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2315), (12225, 2315))

In [151]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_대분류_") ]

train_ft["대분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["대분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["대분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["대분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["대분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["대분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2318), (12225, 2318))

In [152]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_브랜드코드_") ]

train_ft["브랜드코드별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["브랜드코드별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["브랜드코드별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["브랜드코드별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["브랜드코드별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["브랜드코드별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2321), (12225, 2321))

In [153]:
cols = [ col for col in train_ft.columns if col.endswith("요일_구매비율") ]

train_ft["요일별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["요일별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["요일별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["요일별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["요일별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["요일별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2324), (12225, 2324))

In [154]:
cols = [ col for col in train_ft.columns if col.endswith("월_구매비율") ]

train_ft["월별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["월별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["월별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["월별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["월별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["월별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2327), (12225, 2327))

In [155]:
cols = ["봄_구매비율","여름_구매비율","가을_구매비율","겨울_구매비율"]

train_ft["계절별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["계절별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["계절별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["계절별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["계절별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["계절별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2330), (12225, 2330))

In [156]:
cols = [ col for col in train_ft.columns if col.endswith("분기_구매비율") ]

train_ft["분기별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["분기별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["분기별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["분기별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["분기별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["분기별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2333), (12225, 2333))

In [157]:
cols = [ col for col in train_ft.columns if col.endswith("년_구매비율") ]

train_ft["년도별_구매비율_std"] = train_ft[cols].std(axis=1)
test_ft["년도별_구매비율_std"] = test_ft[cols].std(axis=1)

train_ft.shape, test_ft.shape

((14940, 2334), (12225, 2334))

## Feature Encoding

In [158]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

첫구매날짜      338
마지막구매날짜    335
주구매지점        5
주구매_중분류    231
주구매_대분류     29
고객등급         5
dtype: int64

In [165]:
train_ft[train_ft["첫구매날짜"] == '0']

,총방문일수,첫구매날짜,마지막구매날짜,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,평균구매요일,...,월별_구매비율_std,월별_구매비율_skew,월별_구매비율_kurt,계절별_구매비율_std,계절별_구매비율_skew,계절별_구매비율_kurt,분기별_구매비율_std,분기별_구매비율_skew,분기별_구매비율_kurt,년도별_구매비율_std
137,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2594,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [159]:
train_ft["첫구매날짜"] = pd.to_datetime(train_ft["첫구매날짜"])
train_ft["마지막구매날짜"] = pd.to_datetime(train_ft["마지막구매날짜"])
test_ft["첫구매날짜"] = pd.to_datetime(test_ft["첫구매날짜"])
test_ft["마지막구매날짜"] = pd.to_datetime(test_ft["마지막구매날짜"])

train_ft["첫구매년도"] = train_ft["첫구매날짜"].dt.year
train_ft["첫구매월"] = train_ft["첫구매날짜"].dt.month
train_ft["마지막구매년도"] = train_ft["마지막구매날짜"].dt.year
train_ft["마지막구매월"] = train_ft["마지막구매날짜"].dt.month

test_ft["첫구매년도"] = test_ft["첫구매날짜"].dt.year
test_ft["첫구매월"] = test_ft["첫구매날짜"].dt.month
test_ft["마지막구매년도"] = test_ft["마지막구매날짜"].dt.year
test_ft["마지막구매월"] = test_ft["마지막구매날짜"].dt.month

train_ft.shape, test_ft.shape

ValueError: time data "0" doesn't match format "%Y-%m-%d", at position 75. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [289]:
# %pip install category_encoders

In [60]:
# import category_encoders as ce

In [115]:
from sklearn.preprocessing import OneHotEncoder

oh_enc = OneHotEncoder(handle_unknown="ignore")
train_ft[oh_enc.get_feature_names_out()] = oh_enc.fit_transform(train_ft[["주구매지점","주구매_대분류"]]).toarray()
test_ft[oh_enc.get_feature_names_out()] = oh_enc.transform(test_ft[["주구매지점","주구매_대분류"]]).toarray()

train_ft.shape, test_ft.shape

((14940, 2368), (12225, 2368))

In [125]:
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder(categories=[['Normal', 'Black', 'Gold', 'Platinum', 'Diamond']])
train_ft["고객등급"] = ord_enc.fit_transform(train_ft[["고객등급"]])
test_ft["고객등급"] = ord_enc.transform(test_ft[["고객등급"]])

train_ft.shape, test_ft.shape

((14940, 2311), (12225, 2311))

In [135]:
train_ft[train_ft["고객등급"] == 0]["총구매금액"]

0         4201200
1         5043025
3        35594762
4         3988866
5        10047455
           ...   
14933     5268152
14935     1195120
14936     2220000
14937      897898
14938     4136600
Name: 총구매금액, Length: 12625, dtype: int64

In [68]:
enc = ce.count.CountEncoder()
train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft.shape, test_ft.shape

((14940, 2377), (12225, 2377))

- 문자열 피처 삭제

In [80]:
cols = cols + train_ft.select_dtypes("datetime").columns.tolist()
cols

['주구매지점', '주구매_중분류', '주구매_대분류', '고객등급', '첫구매날짜', '마지막구매날짜']

In [82]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 2371), (12225, 2371))

In [83]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

In [84]:
train_ft.select_dtypes("datetime").columns, test_ft.select_dtypes("datetime").columns

(Index([], dtype='object'), Index([], dtype='object'))

## Feature Scaling

In [85]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [89]:
train_ft.isin([np.inf, -np.inf]).any().sum(), test_ft.isin([np.inf, -np.inf]).any().sum()

(0, 0)

In [90]:
train_ft.isna().sum().sum(), test_ft.isna().sum().sum()


(0, 0)

In [91]:
# train_ft = train_ft.replace([np.inf, -np.inf], np.nan)
# test_ft = test_ft.replace([np.inf, -np.inf], np.nan)

# train_ft = train_ft.fillna(0)
# test_ft = test_ft.fillna(0)

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,총방문일수,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,마지막구매후_경과일,평균구매요일,주구매요일,월요일_구매비율,...,주구매_대분류_25,주구매_대분류_26,주구매_대분류_27,주구매_대분류_28,고객등급_1,고객등급_2,고객등급_3,고객등급_4,고객등급_5,주구매_중분류_cnt
0,-0.442360,-0.032407,0.192444,-0.363995,-0.285092,0.019982,0.785722,0.041647,-0.468522,-0.341447,...,-0.090737,-0.127775,-0.084533,-0.038402,0.428213,-0.201153,-0.128585,-0.300321,-0.131775,-0.806829
1,0.202134,0.860877,-0.405966,-0.569485,0.481886,-0.544892,-0.670510,-0.798213,0.118801,-0.503098,...,-0.090737,-0.127775,-0.084533,-0.038402,0.428213,-0.201153,-0.128585,-0.300321,-0.131775,-0.662367
2,2.028201,0.897713,-0.904641,1.485416,0.168122,-0.888460,-0.656092,-0.785470,-1.643170,0.938828,...,-0.090737,-0.127775,-0.084533,-0.038402,-2.335287,4.971346,-0.128585,-0.300321,-0.131775,-0.917799
3,3.907976,0.796413,-1.054243,3.129336,0.577758,-0.986397,-0.569583,0.100820,0.706125,-0.275913,...,-0.090737,-0.127775,-0.084533,-0.038402,0.428213,-0.201153,-0.128585,-0.300321,-0.131775,1.409084
4,0.416966,0.501722,-0.605436,0.457965,-0.484507,-0.536488,-0.079366,1.021483,0.706125,-0.260622,...,-0.090737,-0.127775,-0.084533,-0.038402,0.428213,-0.201153,-0.128585,-0.300321,-0.131775,0.098431


# 정답 데이터

In [92]:
target = train_target["target"]
target

0        1.0
1        1.0
2        0.0
3        0.0
4        0.0
        ... 
14935    0.0
14936    0.0
14937    0.0
14938    1.0
14939    1.0
Name: target, Length: 14940, dtype: float64

# cv 점수 확인해보기

In [93]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

In [94]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=SEED)
scores = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1)
np.mean(scores)

0.719877960713841

# 모델 학습

In [301]:
model = LGBMClassifier(random_state=SEED)
model.fit(train_ft,target)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.036776 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 44036
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 1656
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995


LGBMClassifier(random_state=42)

# 테스트 데이터 예측

In [302]:
pred = model.predict(test_ft)
pred

array([0., 0., 0., ..., 0., 0., 0.])

In [303]:
# pred = model.predict_proba(test_ft)[:,1]
# pred

# 평가를 위한 제출 파일 생성
- 예측 결과를 target 컬럼에 넣어 csv 파일로 저장후에 제출한다.

In [304]:
submit

,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5
...,...,...
12220,test_12220,0.5
12221,test_12221,0.5
12222,test_12222,0.5
12223,test_12223,0.5


In [305]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.0
1,test_1,0.0
2,test_2,0.0
3,test_3,1.0
4,test_4,1.0
...,...,...
12220,test_12220,1.0
12221,test_12221,0.0
12222,test_12222,0.0
12223,test_12223,0.0


- 예측 결과를 csv 파일로 저장하여 제출

In [306]:
submit.to_csv(f"{DATA_PATH}submit.csv",index=False)